In [172]:
import numpy as np
import matplotlib.pyplot as plt
import itertools as itr
import time
from boundaries import WallBoundary

# Initializing a rank three tensor filled with nodes
latticeSize = 9
xResolution = 5
yResolution = 5
relaxationTime = 0.5
# Weights
unitVect = np.array([
	[0, 0],	[1, 0], [0, 1],
	[-1, 0], [0, -1], [1, 1],
	[-1, 1], [-1, -1], [1, -1]
])
unitX = np.array([
	0, 1, 0, -1, 0, 1, -1, -1, 1
])
unitY = np.array([
	0, 0, 1, 0, -1, 1, 1, -1, -1
])
weight = np.array([
    4/9, 1/9, 1/9,
    1/9, 1/9, 1/36,
    1/36, 1/36, 1/36
])
cs = np.sqrt(3)

# Initializing the fluid matrix
fluid = np.ones((xResolution, yResolution, latticeSize)) + 0.1 * np.random.randn(xResolution, yResolution, latticeSize)
# Assigning a right velocity
fluid[:, :, 1] = 2.3

# Set a boundary
boundary = WallBoundary(xResolution, yResolution, False)
boundary.cylindricalWall([xResolution/2, yResolution/2], 3)

In [173]:
# One iteration only
iteratedFluid = np.ones((xResolution, yResolution, latticeSize))

# Internal collision step
density = np.sum(fluid, axis = 2)
mesoMomentum = fluid.reshape(xResolution, yResolution, latticeSize, 1) * unitVect
momentum = np.sum(mesoMomentum, axis = 2) / density.reshape(xResolution, yResolution, 1)
momentumMagnitude = np.linalg.norm(momentum, axis = 2) ** 2 / (2 * cs**4) # Represents u . u

# I kinda give up on broadcasting rn. Help me :skull:
for xIndex, yIndex in itr.product(range(xResolution), range(yResolution)):
    latticeDensity = density[xIndex, yIndex]
    cellMomentumMagnitude = momentumMagnitude[xIndex, yIndex]
    for latticeIndex in range(latticeSize):
        momentumDot = np.dot(unitVect[latticeIndex], momentum[xIndex, yIndex])
        fluidEquilibrium = latticeDensity * weight[latticeIndex] * (
			1 + momentumDot / (cs ** 2) + momentumDot ** 2 / (cs ** 4) - cellMomentumMagnitude
		)
        iteratedFluid[xIndex, yIndex, latticeIndex] = fluid[xIndex, yIndex, latticeIndex] * (1 - 1/relaxationTime) + 1/relaxationTime * fluidEquilibrium

iteratedFluid

array([[[ 8.36721208,  0.15996762,  1.35478867,  1.28146533,
          1.23070061, -0.50866737, -0.54360915, -0.36810871,
         -0.402331  ],
        [ 7.99840498,  0.09563913,  1.49966843,  1.26115149,
          1.30672481, -0.3770395 , -0.42022539, -0.55313755,
         -0.50997428],
        [ 8.51264747,  0.13125695,  1.30190947,  1.29420053,
          1.35004714, -0.33138052, -0.54464714, -0.62333593,
         -0.52546017],
        [ 7.68287601, -0.01816045,  1.1913787 ,  1.0118953 ,
          1.3670822 , -0.38990852, -0.36932011, -0.33090898,
         -0.355202  ],
        [ 8.21703833,  0.05693776,  1.22046191,  1.11448778,
          1.40385211, -0.4447549 , -0.48217916, -0.52302111,
         -0.31366428]],

       [[ 8.07987726,  0.06848868,  1.22219711,  1.3122536 ,
          1.28880746, -0.49587706, -0.53323913, -0.41577839,
         -0.39994078],
        [ 8.00845474,  0.05845385,  1.34398191,  1.17162393,
          1.3085355 , -0.39174402, -0.46230752, -0.48867544,
      

In [175]:
for latticeIndex, shiftX, shiftY in zip(range(latticeSize), unitX, unitY):
    iteratedFluid[:, :, latticeIndex] = np.roll(iteratedFluid[:, :, latticeIndex], shift = shiftX, axis = 1)
    iteratedFluid[:, :, latticeIndex] = np.roll(iteratedFluid[:, :, latticeIndex], shift = shiftY, axis = 0)

fluidBoundary = boundary.boundary
boundaryFluid = iteratedFluid[fluidBoundary, :]

boundaryFluid.

(24, 9)